In [2]:
from adiaframe.utils import *
from adiaframe import Hamiltonian

In [3]:
import pennylane as qml
import numpy as np

from adiaframe.utils import xz_fam_code_add, xz_fam_code_to_pstr, commute_reggio


In [89]:
class PFrame:
    # Schmitz et al, 
    # Optimization at the Interface of Unitary and 
    # Non-unitary Quantum Operations in PCOAST,
    
    def __init__(self, n:int):
        assert n>0 and isinstance(n, int), "n must be a positive integer."
        self.n = n
        num = 1
        frame = []
        for i in range(n):
            frame.append([[0, num], [num, 0]])
            num *= 2
        self.frame = frame
        self.wire_depth = self.n * [0]
        self.gate_log = []
    def supp(self, p):
        num = 0
        for r in self.frame:
            s, sd = r
            a  = not commute_reggio(s, p)
            b  = not commute_reggio(sd, p)
            num+=int(a|b)
        return num
        
    @property
    def str_frame(self):
        frame = []
        for r in self.frame:
            si, sdi = r
            frame.append([xz_fam_code_to_pstr(si,self.n), xz_fam_code_to_pstr(sdi,self.n)])
        return frame
    def reset_gate_log(self):
        self.gate_log = []
    def to_pennylane_circuit(self, replace_rz:None):
        pass
    def CX(self, i, j):
        s_i, sd_i = self.frame[i]
        s_j, sd_j = self.frame[j]
        
        s_j_new = xz_fam_code_add(s_j, s_i)
        sd_i_new = xz_fam_code_add(sd_i, sd_j)
        self.frame[i] = [s_i, sd_i_new]
        self.frame[j] = [s_j_new, sd_j]
        
        self.gate_log.append(["cx", i, j])
        self.wire_depth[i] += 1
        self.wire_depth[j] += 1
    def SWAP(self, i, j):
        nr = self.frame[j]
        self.frame[j] = self.frame[i]
        self.frame[i] = nr
        
        self.gate_log.append(["swap", i, j])
        self.wire_depth[i] += 1
        self.wire_depth[j] += 1
    def H(self, i):
        s_i, sd_i = self.frame[i]
        self.frame[i] = [sd_i, s_i]
        self.gate_log.append(["h", i])
    def S(self, i):
        s_i, sd_i = self.frame[i]
        self.frame[i] = [s_i,  xz_fam_code_add(s_i, sd_i)]
        self.gate_log.append(["s", i])
    def Sd(self, i):
        s_i, sd_i = self.frame[i]
        self.frame[i] = [s_i,  xz_fam_code_add(s_i, sd_i)]
        self.gate_log.append(["sd", i])
    
    def U_mark(self, i):
        self.gate_log.append(["um", i])

In [90]:
# Z family : first consideration.
# 1. start 

0 0 0| 0
0 1 1| 1
1 0 1| 0
1 1 0| 1

(a^b)^a == b

In [91]:
pframe = PFrame(5)
pframe.frame

[[[0, 1], [1, 0]],
 [[0, 2], [2, 0]],
 [[0, 4], [4, 0]],
 [[0, 8], [8, 0]],
 [[0, 16], [16, 0]]]

In [92]:
pframe.str_frame

[['IIIIZ', 'IIIIX'],
 ['IIIZI', 'IIIXI'],
 ['IIZII', 'IIXII'],
 ['IZIII', 'IXIII'],
 ['ZIIII', 'XIIII']]

In [93]:
pframe.CX(0, 3)
pframe.H(0)
pframe.S(0)
pframe.SWAP(2, 4)

In [94]:
pframe.frame

[[[9, 0], [9, 1]],
 [[0, 2], [2, 0]],
 [[0, 16], [16, 0]],
 [[0, 9], [8, 0]],
 [[0, 4], [4, 0]]]

In [95]:
pframe.str_frame

[['IXIIX', 'IXIIY'],
 ['IIIZI', 'IIIXI'],
 ['ZIIII', 'XIIII'],
 ['IZIIZ', 'IXIII'],
 ['IIZII', 'IIXII']]

In [98]:
pframe.supp((3, 2))

3

In [103]:
pframe.supp((2, 5))

3

In [102]:
xz_fam_code_to_pstr((2, 5), 5)

'IIZXZ'